In [ ]:
modelId = 'us.anthropic.claude-3-5-sonnet-20241022-v2:0'

config = {
    "credentials_profile_name": "default",
    "region_name": "us-east-1",
    "llm": modelId,
    "embeddings": "amazon.titan-embed-text-v2:0",
    "temperature": 0.4,
}

In [ ]:
from langchain_aws import ChatBedrockConverse
from langchain_aws import BedrockEmbeddings
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

evaluator_llm = LangchainLLMWrapper(ChatBedrockConverse(
    credentials_profile_name=config["credentials_profile_name"],
    region_name=config["region_name"],
    base_url=f"https://bedrock-runtime.{config['region_name']}.amazonaws.com",
    model=config["llm"],
    temperature=config["temperature"],
))
evaluator_embeddings = LangchainEmbeddingsWrapper(BedrockEmbeddings(
    credentials_profile_name=config["credentials_profile_name"],
    region_name=config["region_name"],
    model_id=config["embeddings"],
))

In [ ]:
import json
from dataset import extract_rag_data

source_data = extract_rag_data(csv_path="questions_file.csv")
dataset = [
    {
        "user_input": item["user_input"],
        "response": item["response"],
        "reference": item["reference"],
        "retrieved_contexts": item["retrieved_contexts"]
    } 
    for item in source_data
]

print(json.dumps(dataset, indent=2, ensure_ascii=False))

In [ ]:
from ragas import EvaluationDataset
eval_dataset = EvaluationDataset.from_list(dataset)
print("Features in dataset:", eval_dataset.features())
print("Total samples in dataset:", len(eval_dataset))

In [ ]:
from ragas import evaluate
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness

result = evaluate(dataset=eval_dataset,metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness()],llm=evaluator_llm)
result

In [15]:
df = result.to_pandas()
df

,user_input,retrieved_contexts,response,reference,context_recall,faithfulness,factual_correctness(mode=f1)
0,כמה ימי חופשה מקבלים בשנה?,"[[הסכם 17.08.2015 סעיף 7.2] ""עם המעבר לשבוע עב...","נכון להיום, בהתאם להסכם משנת 2015, רופא וטרינר...","נכון להיום (2015), רופא וטרינר זכאי ל-22 ימי ח...",0.666667,0.666667,0.73
1,מהם כללי הקידום לרופא וטרינרי שאינו מומחה במשר...,"[[הסכם 10.02.1992 סעיף 4] ""רופא וטרינרי (שאינו...",רופא וטרינרי שאינו מומחה במשרד הבריאות מתחיל ב...,"נכון להיום (2015), כללי הקידום לרופא וטרינרי ש...",0.500000,0.900000,0.77
2,מאיזו שעה משולמת שעה נוספת?,"[[הסכם 17.08.2015 סעיף 6.2] ""**שבוע העבודה יעמ...","החל מיום 1.9.2015, שעה נוספת משולמת לאחר השלמת...","החל מיום 1.9.2015, שעה נוספת משולמת לאחר השלמת...",0.000000,0.000000,0.89
